In [9]:
FOLDERNAME = '/Users/aolyu/Documents/GitHub/QTM-Project/QTM550Team1Plottings 2'

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler, RandomSampler, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from PIL import Image, ImageOps
import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
from sklearn.metrics import confusion_matrix
import time

import seaborn as sns
from __future__ import print_function 
from __future__ import division
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.2.1
Torchvision Version:  0.17.1


In [17]:
BATCH_SIZE = 10

first_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor() ])

train_set = datasets.ImageFolder(root=os.path.join(FOLDERNAME,'train'), transform = first_transform)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 16)

mean = 0.
std = 0.
nb_samples = 0.
for data, labels in train_loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

/Users/aolyu/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 14 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/aolyu/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/aolyu/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aolyu/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 277, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aolyu/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 144, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
           ^^^^^^^^^^^^^^^^^^^^^^^^^

In [11]:
# resize the image(tif) in FOLDERNAME into 224*224
def resize_image(FOLDERNAME):
    for filename in os.listdir(FOLDERNAME):
        if filename.endswith('.tif'):
            img = Image.open(FOLDERNAME + '/' + filename)
            img = img.resize((224, 224))
            img.save(FOLDERNAME + '/' + filename)
    return

resize_image(FOLDERNAME)

# load the image(tif) in FOLDERNAME
def load_image(FOLDERNAME):
    images = []
    for filename in os.listdir(FOLDERNAME):
        if filename.endswith('.tif'):
            img = Image.open(FOLDERNAME + '/' + filename)
            images.append(img)
    return images

images = load_image(FOLDERNAME)   

# transform the image into tensor
def transform_image(images):
    tensor_images = []
    for img in images:
        img = T.ToTensor()(img)
        tensor_images.append(img)
    return tensor_images

tensor_images = transform_image(images)

In [16]:
# show the image(tif) in FOLDERNAME into 224*224
def show_image(tensor_images):
    for img in tensor_images:
        img = img.numpy().transpose((1, 2, 0))
        plt.imshow(img)
        plt.show()
    return

show_image(tensor_images)

# load the label
def load_label(FOLDERNAME):
    label = []
    for filename in os.listdir(FOLDERNAME):
        if filename.endswith('.txt'):
            with open(FOLDERNAME + '/' + filename) as f:
                for line in f:
                    label.append(int(line))
    return label

label = load_label(FOLDERNAME)
show_image(tensor_images)